### Step 1 - Start Spark Session, include additional configurations and common functions, and read data from the spark-warehouse database

In [1]:
%run "../includes/configurations"

In [2]:
%run "../includes/common_functions"

In [3]:
appName = "Lap Times Ingestion"
spark = create_spark_context(appName, warehouse_location)

24/01/04 18:53:28 WARN Utils: Your hostname, falcao-sys resolves to a loopback address: 127.0.1.1; using 192.168.11.185 instead (on interface wlx7898e8c12476)
24/01/04 18:53:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/04 18:53:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
lap_times_df = spark.read.parquet(f"{warehouse_location}/f1_raw.db/lap_times")

### Step 3 - Rename and add new columns

In [5]:
lap_times_column_mapping = {
    "raceId": "race_id",
    "driverId": "driver_id"
}

lap_times_df = rename_columns(lap_times_df, lap_times_column_mapping)

In [6]:
lap_times_df = add_ingestion_date(lap_times_df)

### Step 3 - Write a data to a Datalake as a Parquet

In [7]:
lap_times_df.write.mode("overwrite").parquet(f"{processed_folder_path}/lap_times")